In [130]:
import pandas as pd   
import json
import requests
from pandas import json_normalize

In [131]:
# Options to read a jsonl file: 

#with open('./data/twitter_monkeypox.jsonl', 'r') as json_file:
#    json_list = list(json_file)
#
#for json_str in json_list:
#    result = json.loads(json_str)
   # print(f"result: {result}")
   # print(isinstance(result, dict))
#    print(result['_type'])
    
#
#result = [json.loads(jline) for jline in jsonl_content.splitlines()]
#result = [json.loads(jline) for jline in response.read().splitlines()]    

#with open('/path/to/file.jsonl') as f:
#    data = [json.loads(line) for line in f]

In [132]:
df_raw = pd.read_json(path_or_buf='./data/twitter_monkeypox.jsonl', lines=True)
df_raw.head()


,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,snscrape.modules.twitter.Tweet,https://twitter.com/IMSS_HGO/status/1628385619...,2023-02-22 13:26:00+00:00,🧐 Con medidas preventivas sencillas se puede d...,🧐 Con medidas preventivas sencillas se puede d...,1628385619326955521,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,None,None,None,"[MonkeyPox, ViruelaDelMono]",None,None,4.0,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/OMSCentrafrique/status/162...,2023-02-22 11:43:51+00:00,L'@OMSCentrafrique appuie la formation de 4️⃣8...,L'@OMSCentrafrique appuie la formation de 4️⃣8...,1628359913050189825,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,4,4,...,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,"[santé, Monkeypox, choléra, RCA, Monkeypox]",None,None,184.0,None
2,snscrape.modules.twitter.Tweet,https://twitter.com/Univadis/status/1628356434...,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#Monkeypox remains global health emergency #Me...,1628356434218868738,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,None,16.0,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/sanidadgob/status/16283551...,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,"{'_type': 'snscrape.modules.twitter.User', 'us...",2,2,2,...,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,None,2613.0,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/redaccionmedica/status/162...,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,1628343266893602817,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,2,...,NaN,None,None,None,None,[monkeypox],None,{'_type': 'snscrape.modules.twitter.SummaryCar...,545.0,None


In [133]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             5000 non-null   object             
 1   url               5000 non-null   object             
 2   date              5000 non-null   datetime64[ns, UTC]
 3   rawContent        5000 non-null   object             
 4   renderedContent   5000 non-null   object             
 5   id                5000 non-null   int64              
 6   user              5000 non-null   object             
 7   replyCount        5000 non-null   int64              
 8   retweetCount      5000 non-null   int64              
 9   likeCount         5000 non-null   int64              
 10  quoteCount        5000 non-null   int64              
 11  conversationId    5000 non-null   int64              
 12  lang              5000 non-null   object             
 13  sou

In [134]:
df=df_raw.drop('_type',axis=1)

In [135]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day

In [136]:
df.loc[1].at['url'].split('/')[3]

'OMSCentrafrique'

In [137]:
df['username']=df.url.apply(lambda x: x.split('/')[3])

In [138]:
df.columns

Index(['url', 'date', 'rawContent', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'links',
       'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
       'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags',
       'cashtags', 'card', 'viewCount', 'vibe', 'year', 'month', 'day',
       'username'],
      dtype='object')

In [139]:
df=df.drop('url',axis=1)

In [140]:
df.columns

Index(['date', 'rawContent', 'renderedContent', 'id', 'user', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang',
       'source', 'sourceUrl', 'sourceLabel', 'links', 'media',
       'retweetedTweet', 'quotedTweet', 'inReplyToTweetId', 'inReplyToUser',
       'mentionedUsers', 'coordinates', 'place', 'hashtags', 'cashtags',
       'card', 'viewCount', 'vibe', 'year', 'month', 'day', 'username'],
      dtype='object')

In [141]:
df=df[['username','year', 'month', 'day','date', 'rawContent', 'renderedContent', 'id', 'user', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang',
       'source', 'sourceUrl', 'sourceLabel', 'links', 'media',
       'retweetedTweet', 'quotedTweet', 'inReplyToTweetId', 'inReplyToUser',
       'mentionedUsers', 'coordinates', 'place', 'hashtags', 'cashtags',
       'card', 'viewCount', 'vibe']]


In [142]:
df.loc[1].at['user'].keys()

dict_keys(['_type', 'username', 'id', 'displayname', 'rawDescription', 'renderedDescription', 'descriptionLinks', 'verified', 'created', 'followersCount', 'friendsCount', 'statusesCount', 'favouritesCount', 'listedCount', 'mediaCount', 'location', 'protected', 'link', 'profileImageUrl', 'profileBannerUrl', 'label', 'url'])

In [143]:
df['user'][1]

{'_type': 'snscrape.modules.twitter.User',
 'username': 'OMSCentrafrique',
 'id': 800648249241522176,
 'displayname': 'OMS Centrafrique',
 'rawDescription': 'Organisation Mondiale de la Santé en République Centrafricaine #RCA | World Health Organization in Central African Republic 🇨🇫',
 'renderedDescription': 'Organisation Mondiale de la Santé en République Centrafricaine #RCA | World Health Organization in Central African Republic 🇨🇫',
 'descriptionLinks': None,
 'verified': True,
 'created': '2016-11-21T10:33:12+00:00',
 'followersCount': 6371,
 'friendsCount': 224,
 'statusesCount': 979,
 'favouritesCount': 959,
 'listedCount': 123,
 'mediaCount': 486,
 'location': 'Bangui, RCA',
 'protected': False,
 'link': {'_type': 'snscrape.modules.twitter.TextLink',
  'text': 'afro.who.int/fr/countries/c…',
  'url': 'https://www.afro.who.int/fr/countries/central-african-republic',
  'tcourl': 'https://t.co/IPvLV4A9zA',
  'indices': [0, 23]},
 'profileImageUrl': 'https://pbs.twimg.com/profile_i

In [144]:
df.loc[1].at['user']['followersCount']

6371

In [145]:
df.loc[1,'user']['username']

'OMSCentrafrique'

In [146]:
df['displayname']=df.user.apply(lambda x:x['displayname'])

In [147]:
df['followerscount']=df.user.apply(lambda x:x['followersCount'])

In [148]:
df['friendscount']=df.user.apply(lambda x:x['friendsCount'])

In [149]:
df['location']=df.user.apply(lambda x:x['location'])

In [150]:
df['mediacount']=df.user.apply(lambda x:x['mediaCount'])

In [151]:
df.columns

Index(['username', 'year', 'month', 'day', 'date', 'rawContent',
       'renderedContent', 'id', 'user', 'replyCount', 'retweetCount',
       'likeCount', 'quoteCount', 'conversationId', 'lang', 'source',
       'sourceUrl', 'sourceLabel', 'links', 'media', 'retweetedTweet',
       'quotedTweet', 'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers',
       'coordinates', 'place', 'hashtags', 'cashtags', 'card', 'viewCount',
       'vibe', 'displayname', 'followerscount', 'friendscount', 'location',
       'mediacount'],
      dtype='object')

In [118]:
df=df.drop('user',axis=1)

In [119]:
df=df.drop('id',axis=1)

In [128]:
df.head()

,username,displayname,year,month,day,date,rawContent,renderedContent,replyCount,retweetCount,...,coordinates,place,hashtags,cashtags,card,viewCount,vibe,followerscount,friendscount,location
0,IMSS_HGO,IMSS Hidalgo,2023,2,22,2023-02-22 13:26:00+00:00,🧐 Con medidas preventivas sencillas se puede d...,🧐 Con medidas preventivas sencillas se puede d...,0,0,...,None,None,"[MonkeyPox, ViruelaDelMono]",None,None,4.0,None,3164,269,"Hidalgo, Pachuca de Soto"
1,OMSCentrafrique,OMS Centrafrique,2023,2,22,2023-02-22 11:43:51+00:00,L'@OMSCentrafrique appuie la formation de 4️⃣8...,L'@OMSCentrafrique appuie la formation de 4️⃣8...,1,4,...,None,None,"[santé, Monkeypox, choléra, RCA, Monkeypox]",None,None,184.0,None,6371,224,"Bangui, RCA"
2,Univadis,Univadis,2023,2,22,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#Monkeypox remains global health emergency #Me...,0,0,...,None,None,"[Monkeypox, MedTwitter]",None,None,16.0,None,626,997,
3,sanidadgob,Ministerio de Sanidad,2023,2,22,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,2,2,...,None,None,"[ViruelaDelMono, Monkeypox]",None,None,2613.0,None,699628,695,"Madrid, España"
4,redaccionmedica,Redacción Médica,2023,2,22,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,0,0,...,None,None,[monkeypox],None,{'_type': 'snscrape.modules.twitter.SummaryCar...,545.0,None,97401,7730,España


In [84]:
#df2 = pd.json_normalize(df['user'])
#df2

,_type,username,id,displayname,rawDescription,renderedDescription,descriptionLinks,verified,created,followersCount,...,link.text,link.url,link.tcourl,link.indices,link,label._type,label.description,label.url,label.badgeUrl,label.longDescription
0,snscrape.modules.twitter.User,IMSS_HGO,842855141674024961,IMSS Hidalgo,#IMSS Hidalgo,#IMSS Hidalgo,None,False,2017-03-17T21:48:20+00:00,3164,...,imss.gob.mx,http://www.imss.gob.mx,https://t.co/CNTtGnotQn,"[0, 23]",NaN,NaN,NaN,NaN,NaN,NaN
1,snscrape.modules.twitter.User,OMSCentrafrique,800648249241522176,OMS Centrafrique,Organisation Mondiale de la Santé en Républiqu...,Organisation Mondiale de la Santé en Républiqu...,None,True,2016-11-21T10:33:12+00:00,6371,...,afro.who.int/fr/countries/c…,https://www.afro.who.int/fr/countries/central-...,https://t.co/IPvLV4A9zA,"[0, 23]",NaN,NaN,NaN,NaN,NaN,NaN
2,snscrape.modules.twitter.User,Univadis,3640708933,Univadis,"Univadis brings you unbiased, relevant medical...","Univadis brings you unbiased, relevant medical...",None,False,2015-09-21T19:00:36+00:00,626,...,univadis.com,http://www.univadis.com,http://t.co/LjAkdKTAp4,"[0, 22]",NaN,NaN,NaN,NaN,NaN,NaN
3,snscrape.modules.twitter.User,sanidadgob,232124120,Ministerio de Sanidad,Ministerio de Sanidad #YoMeVacunoSeguro,Ministerio de Sanidad #YoMeVacunoSeguro,None,True,2010-12-30T09:00:21+00:00,699628,...,sanidad.gob.es,https://www.sanidad.gob.es/,https://t.co/Uz115LXvLp,"[0, 23]",NaN,NaN,NaN,NaN,NaN,NaN
4,snscrape.modules.twitter.User,redaccionmedica,229001251,Redacción Médica,Periódico online con toda la actualidad del se...,Periódico online con toda la actualidad del se...,None,True,2010-12-21T07:37:42+00:00,97401,...,redaccionmedica.com,http://www.redaccionmedica.com,https://t.co/EEsU0Jg8HV,"[0, 23]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,snscrape.modules.twitter.User,Geovax_News,2198018796,"GeoVax, Inc.",GeoVax is a biotechnology company developing v...,GeoVax is a biotechnology company developing v...,None,False,2013-11-16T16:29:00+00:00,2415,...,geovax.com,http://www.geovax.com,https://t.co/J3tDwBG6BW,"[0, 23]",NaN,NaN,NaN,NaN,NaN,NaN
4996,snscrape.modules.twitter.User,abplive,333888813,ABP LIVE,Follow @ABPLive for breaking news alerts and l...,Follow @ABPLive for breaking news alerts and l...,None,False,2011-07-12T07:23:31+00:00,2093659,...,news.abplive.com,https://news.abplive.com/,https://t.co/yjIOTq0zvJ,"[0, 23]",NaN,NaN,NaN,NaN,NaN,NaN
4997,snscrape.modules.twitter.User,Mott_Mason,918813768418775040,Mott Mason,Politically adventurous. Freedom & Direct Demo...,Politically adventurous. Freedom & Direct Demo...,None,False,2017-10-13T12:20:47+00:00,3819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,snscrape.modules.twitter.User,br00t4c,1330911446440554496,🌊 Peter Taylor 💙 🌎,Salty commentary all day. Opinionated. Woke. #...,Salty commentary all day. Opinionated. Woke. #...,None,False,2020-11-23T16:30:57+00:00,6964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#df = pd.concat([df, df2], axis=1)
#df = df.drop('user', axis=1)